https://tutorials.pytorch.kr/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

## Build Neural Networks

`torch.nn`은 모델을 정의하고 미분하는데 `autograd`를 사용하며, `nn.Module()`은 layer와 output을 반환하는 `forward(input)`메서드를 포함하고 있다.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb

wandb.init(project='wonchul')

W&B Run: https://app.wandb.ai/onedang2/wonchul/runs/17voctd3

다음과 같은 network를 구성하겠다.

<img src='./imgs/nn.png'>

> nn.Conv2d(input channel, output channel, filter size)

신경망의 일반적인 학습 과정은 다음과 같습니다:

* 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의합니다.

* 데이터셋(dataset) 입력을 반복합니다.

* 입력을 신경망에서 전파(process)합니다.

* 손실(loss; 출력이 정답으로부터 얼마나 떨어져있는지)을 계산합니다.

* 변화도(gradient)를 신경망의 매개변수들에 역으로 전파합니다.

* 신경망의 가중치를 갱신합니다. 일반적으로 다음과 같은 간단한 규칙을 사용합니다: <br/>`가중치(wiehgt) = 가중치(weight) - 학습율(learning rate) * 변화도(gradient)`


In [6]:
class network(nn.Module):
    def __init__(self):
        super(network, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 3) 
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # max pooling over a 2x2 window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimension except the batch dimension
        
        nb_features = 1        
        for s in size:
            nb_features *= s

        return nb_features

> input shape: 1x32x32

> conv1 output: (32 - 3 + 2*0)/1 + 1 = 30 >> 6x30x30

> pooling output: 6x15x15

> conv2 output: (15 - 3 + 2*0)/1 + 1 = 13 >> 16x13x13

> pooling output: 16x6x6

In [7]:
net = network()
net

network(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

`forward` 함수만 정의하고 나면, (변화도를 계산하는) `backward` 함수는 `autograd` 를 사용하여 자동으로 정의됩니다. `forward` 함수에서는 어떠한 Tensor 연산을 사용해도 됩니다.

모델의 학습 가능한 매개변수들은 `net.parameters()` 에 의해 반환됩니다.

In [8]:
wandb.watch(net)

In [9]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 3, 3])


임의의 32x32 입력값을 넣어보겠습니다.

In [10]:
input = torch.randn(1, 1, 32, 32)
output = net(input)
print(output, output.size())

tensor([[ 0.0383, -0.0223, -0.0314, -0.0190,  0.0679,  0.0106,  0.0619, -0.0101,
         -0.0911, -0.0022]], grad_fn=<AddmmBackward>) torch.Size([1, 10])


(1x32x32 - 6x3 + 2*0)/1 + 1 = 6x30x30
(6x30x30 - 6x2x2 + 0)/1 + 1 = 6x29x29
(6x29x29 - 16x3x3 + 0)/1 + 1 = 12




모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로 역전파를 합니다:



In [11]:
net.zero_grad()
output.backward(torch.randn(1, 10))

torch.nn 은 미니-배치(mini-batch)만 지원합니다. torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니-배치만을 입력으로 받습니다.

예를 들어, nnConv2D 는 nSamples x nChannels x Height x Width 의 4차원 Tensor를 입력으로 합니다.

만약 하나의 샘플만 있다면, input.unsqueeze(0) 을 사용해서 가짜 차원을 추가합니다.

torch.Tensor - backward() 같은 autograd 연산을 지원하는 다차원 배열 입니다. 또한 tensor에 대한 변화도(gradient)를 갖고 있습니다.

nn.Module - 신경망 모듈. 매개변수를 캡슐화(encapsulation)하는 간편한 방법 으로, GPU로 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공합니다.

nn.Parameter - Tensor의 한 종류로, Module 에 속성으로 할당될 때 자동으로 매개변수로 등록 됩니다.

autograd.Function - autograd 연산의 전방향과 역방향 정의 를 구현합니다. 모든 Tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor 를 생성하고 이력(history)을 부호화 하는 함수들과 연결하고 있습니다.

In [16]:
output_ = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
wandb.log('loss':loss)

tensor(0.8527, grad_fn=<MseLossBackward>)

In [18]:
loss.grad_fn

이제 .grad_fn 속성을 사용하여 loss 를 역방향에서 따라가다보면, 이러한 모습의 연산 그래프를 볼 수 있습니다:



input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

따라서 loss.backward() 를 실행할 때, 전체 그래프는 손실(loss)에 대하여 미분되며, 그래프 내의 requires_grad=True 인 모든 Tensor는 변화도(gradient)가 누적된 .grad Tensor를 갖게 됩니다.

In [19]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU


오차(error)를 역전파하기 위해서는 loss.backward() 만 해주면 됩니다. 기존 변화도를 없애는 작업이 필요한데, 그렇지 않으면 변화도가 기존의 것에 누적되기 때문입니다.

In [20]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [21]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)